# Uczenie maszynowe z wykorzystaniem szyfrowania homomorficznego
### Autorzy: Marcin Szwed, Konrad Meling, Patryk Twardowski

### Wprowadzenie do biblioteki TenSEAL

https://github.com/OpenMined/TenSEAL/tree/main

TenSEAL to biblioteka do homomorficznego szyfrowania danych w Pythonie. Umożliwia wykonywanie operacji arytmetycznych na zaszyfrowanych danych, z zachowaniem ich poufności. Zbudowana jest na bazie biblioteki Microsoft SEAL, zapewniając wysokopoziomowe API przy zachowaniu wydajności operacji.

Microsoft SEAL, na której bazuje TenSEAL, to biblioteka C++ rozwijana przez Cryptography Research Group firmy Microsoft. Umożliwia generowanie kluczy, szyfrowanie i deszyfrowanie przy użyciu popularnych schematów homomorficznych, takich jak CKKS (dla danych zmiennoprzecinkowych) oraz BFV (dla liczb całkowitych). Użycie SEAL wymaga znajomości zagadnień kryptograficznych i zarządzania pamięcią, co czyni TenSEAL wygodnym, uproszczonym interfejsem dla użytkowników Pythona.

TenSEAL upraszcza procedurę szyfrowania, udostępniając wygodne API z niższym progiem wejścia. Biblioteka pozwala na enkrypcję i dekrypcję używając BFV i CKKS.

Podstawowe funkcjonalności:
- dodawanie: sum_inplace, sum_batch_inplace
- mnożenie macierzy: matmul_inplace, matmul_plain_inplace
- iloczyn skalarny: dot, dot_inplace
- potęgowanie: power_inplace
- negacja: negate_inplace

TenSEAL wspiera także operacje na tensorach a także konwolucję, jest kompatybilna z biblioteką TensorFlow i PyTorch co sprawia że można jej używać w zastosowanych zadaniach dotyczących sieci neuronowych. 

Ze względu na zastosowanie operacji homomorficznych, nie jest możliwe obliczanie funkcji nieliniowych, takich jak sigmoid, a co za tym idzie stosujemy przybliżenia wielomianowe, np:

sigmoid(x) ≈ 0.5 + 0.197·x − 0.004·x³

### Krótkie przedstawienie możliwości biblioteki

Tworzymy kontekst CKKS dla szyfrowania homomorficznego, używając następujących parametrów:
 - poly_modulus_degree - stopień wielomianu pierścieniowego, wpłwa na bezpieczeństwo i czas wykonywania operacji
 - coeff_mod_bit_size - wskazuje poziomy redukcji szumu podczas szyfrowania, ilość modułów określa możliwe operacje mnożenia - w naszy przypadku 3
 - global_scale - globalny współczynnik skalujący. Ckks używa go do kontrolowania precyzji liczb zmiennoprzecinkowych, im wyższa wartość tym większa dokładność, ale istnieje ryzyko błędu

In [3]:
import tenseal as ts
context = ts.context(ts.SCHEME_TYPE.CKKS, 
                    poly_modulus_degree=8192, 
                    coeff_mod_bit_sizes=[60, 40, 40, 60] )
context.global_scale = 2**40

Generujemy klucze Galois, potrzebne do rotacji wektorów zaszyfrowanych w homomorficznym szyfrowaniu

In [4]:
context.generate_galois_keys()

Przeprowadzamy proste testy

In [5]:
vector1 = [0.1, 0.2, 0.3, 0.4, 0.5]
vector2 = [1, 2, 3, 4, 5]
print(f"wektor 1: {vector1}")
print(f"wektor 2: {vector2} \n")
encrypted_vector1 = ts.ckks_vector(context, vector1)
encrypted_vector2 = ts.ckks_vector(context, vector2)

decrypted_vector = encrypted_vector1.decrypt()
print(f"dekrypcja wektora 1: {decrypted_vector} \n")

add_result = encrypted_vector1 + [1, 2, 3, 4, 5]
print(f"wynik dodawania wektora zakodowanego i jawnego: {add_result.decrypt()}") 

encrypted_sum = encrypted_vector1 + encrypted_vector2
print(f"wynik dodawania dwóch wektorów zakodowanych: {encrypted_sum.decrypt()} \n") 

sub_result = encrypted_vector1 - [1, 2, 3, 4, 5]
print(f"wynik odejmowania wektora zakodowanego i jawnego: {sub_result.decrypt()}") 

encrypted_diff = encrypted_vector1 - encrypted_vector2
print(f"wynik odejmowania dwóch wektorów zakodowanych: {encrypted_diff.decrypt()}\n") 

mul_result = encrypted_vector1 * [1, 2, 3, 4, 5]
print(mul_result.decrypt())  
print(f"wynik mnożenia wektora zakodowanego przez jawny: {mul_result.decrypt()}") 

encrypted_product = encrypted_vector1 * encrypted_vector2
print(f"wynik mnożenia dwóch wektorów zakodowanych: {encrypted_product.decrypt()}") 

wektor 1: [0.1, 0.2, 0.3, 0.4, 0.5]
wektor 2: [1, 2, 3, 4, 5] 

dekrypcja wektora 1: [0.10000000018142119, 0.2000000018217, 0.2999999995825561, 0.4000000008859689, 0.4999999997796727] 

wynik dodawania wektora zakodowanego i jawnego: [1.1000000001771122, 2.2000000018433807, 3.2999999996024583, 4.4000000008897455, 5.499999999776742]
wynik dodawania dwóch wektorów zakodowanych: [1.1000000010466078, 2.2000000030882014, 3.3000000002861603, 4.400000000935252, 5.500000000324521] 

wynik odejmowania wektora zakodowanego i jawnego: [-0.89999999981427, -1.7999999981999804, -2.7000000004373463, -3.5999999991178075, -4.500000000217398]
wynik odejmowania dwóch wektorów zakodowanych: [-0.9000000006837658, -1.7999999994448008, -2.7000000011210483, -3.5999999991633143, -4.500000000765176]

[0.10000001355017132, 0.40000005152148, 0.9000001190189988, 1.6000002176984236, 2.5000003319328745]
wynik mnożenia wektora zakodowanego przez jawny: [0.10000001355017132, 0.40000005152148, 0.9000001190189988, 1.600

Przykładowa estymacja funkcji sigmoid(x) wielomianem trzeciego stopnia dla zakodowanego wektora. Z racji iż mamy do czynienia z wielomianem trzeciego stopnia, czyli kilkoma mnożeniami, następuje widoczna utrata dokładności

In [6]:
from scipy.special import expit

result = encrypted_vector1.polyval([0.5, 0.197, 0, -0.004])

y = expit(vector1)

print(f"sigmoid dla wektora jawnego: {y}")
print(f"sigmoid dla wektora zakodowanego: {result.decrypt()}")

sigmoid dla wektora jawnego: [0.52497919 0.549834   0.57444252 0.59868766 0.62245933]
sigmoid dla wektora zakodowanego: [0.5196960021808914, 0.5393680061507066, 0.5589920080900813, 0.5785440112288189, 0.5980000120385917]


TenSEAL wspiera także:
 - serializację i deserializację obiektów (save, load)
 - operacje im2col dla konwolucji
 - szyfrowane mnożenie macierz-wektor

## Przegląd algorytmów
* regresja liniowa, metody gradientowe - stosunkowo proste, wyłącznie mnożenie i dodawanie
* pełna liniowa regresja - jeżeli serwer zgromadziłby wszystkie dane od klientów konieczne jest odwracanie macierzy
* regresja z użyciem prostych sieci neuronowych np. z jedną warstwą ukrytą - teoretycznie możliwa, ale zawyczaj stosowane są nieliniowe funkcje aktywacji jak ReLU
* regresja logistyczna - przy wyznaczaniu prawdopodobieństwa jest stosowana funkcja sigmoidalna, którą można w pewnym zakresie wartości aproksymować wielomianem
* klasyfikacja naiwną metodą Bayesa - wymaga funkcji log(), chyba, że mały wymiar, bo wtedy można pomnożyć prawdopodobieństwa
* klasyfikacja metodą najbliższysch sąsiadów KNN - teoretycznie można obliczać w zaszyfrowany sposób odległość, ale musiałaby być porównywana po stronie klienta więc raczej bez sensu
* drzewa decyzyjne - wymagają porównań
* klasyfikacja z siecią neuronową - wymaga exp() w ostatniej warstwie, bo sigmoid albo softmax jest funkcją aktywacji ostatniej warstwy, funkcja celu zawiera zazwyczaj logarytm (cross entropy)

## Przypadki użycia

Scenariusz, w którym serwer przechowuje model niezaszyfrowany.

* Klient przesyła do serwera zaszyfrowane dane oraz klucz publiczny
* Serwer szyfruje nim model
* Serwer przeprowadza predykcję używając zaszyfrowanych parametrów modelu i zaszyfrowanych danych
* Serwer zwraca wyniki predykcji
* Klient odszyfrowuje wyniki predykcji posługując się kluczem prywatnym

Scenariusz, w którym serwer otrzymuje zaszyfrowane dane i trenuje model z zaszyfrowanymi parametrami.

* Klient wysyła zaszyfrowane dane i klucz publiczny do serwera, przesyła także informacje o danych typu rozmiar danych i liczba rekordów
* Serwer inicjuje model i szyfruje go
* Następnie w pętli serwer wykonuje kolejne kroki uczenia, podczas których wyznacza funkcję straty, jej gradient względem wag, a następnie uaktualnia wagi
* Po zakończeniu treningu przesyła do klienta zaszyfrowane wagi lub cały zaszyfrowany model
* Klient odszyfrowuje wyniki, może ocenić jakość modelu i użyć go do predykcji na własnych danych



## Poniższy przykład
* klient ma parametry modelu i dane
* klient przesyła zaszyfrowane parametry modelu i dane do serwera
* serwer oblicza błąd i zwraca do klienta
* klient uaktualnia model

In [1]:
import tenseal as ts
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

data = load_diabetes()
X = data.data[:, :3]  
y = data.target

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

X_train = X[:20]
y_train = y[:20]

context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60]
)
context.global_scale = 2 ** 40
context.generate_galois_keys()


n_features = X_train.shape[1]
w = np.zeros(n_features)
b = 0.0
learning_rate = 0.1
epochs = 10

#Trening: klient-serwer
for epoch in range(epochs):
    grad_w = np.zeros(n_features)
    grad_b = 0.0

    # klient szyfruje parametry
    w_enc = ts.ckks_vector(context, w.tolist())
    b_enc = ts.ckks_vector(context, [b])

    
    for i in range(len(X_train)):
        x_i = X_train[i]
        y_i = y_train[i]
        # klient szyfruje dane
        x_enc = ts.ckks_vector(context, x_i.tolist())
        y_enc = ts.ckks_vector(context, [y_i])

        # Tu oblicza serwer
        pred_enc = x_enc.dot(w_enc) + b_enc

        error_enc = pred_enc - y_enc

        # serwer wysyła do klienta
        error = error_enc.decrypt()[0]

        grad_w += error * x_i
        grad_b += error

    grad_w /= len(X_train)
    grad_b /= len(X_train)

    w -= learning_rate * grad_w
    b -= learning_rate * grad_b

    y_pred = X_train.dot(w) + b
    mse = mean_squared_error(y_train, y_pred)
    print(f"Epoka {epoch+1}, MSE (HE): {mse:.5f}")


lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_train)
mse_lr = mean_squared_error(y_train, y_pred_lr)

print("MSE modelu HE: ", mean_squared_error(y_train, X_train.dot(w) + b))
print("MSE modelu sklearn: ", mse_lr)
print("Wagi HE:", w)
print("Wagi sklearn: ", lr.coef_)
print("Bias HE: ", b)
print("Bias sklearn: ", lr.intercept_)

Epoka 1, MSE (HE): 0.51257
Epoka 2, MSE (HE): 0.49286
Epoka 3, MSE (HE): 0.47592
Epoka 4, MSE (HE): 0.46118
Epoka 5, MSE (HE): 0.44824
Epoka 6, MSE (HE): 0.43677
Epoka 7, MSE (HE): 0.42657
Epoka 8, MSE (HE): 0.41743
Epoka 9, MSE (HE): 0.40922
Epoka 10, MSE (HE): 0.40182
MSE modelu HE:  0.4018181815793052
MSE modelu sklearn:  0.324313493798384
Wagi HE: [-0.14735328 -0.06858322  0.17461548]
Wagi sklearn:  [-0.32609907 -0.13320203  0.55650174]
Bias HE:  -0.09468727161194786
Bias sklearn:  -0.13157153183388606
